# Train a model for song similarity

In [ ]:
__import__('sys').path.append('../scripts/'); __import__('notebook_utils').table_of_contents('song_similarity_training.ipynb')

Steps:
1. **Get matching songs** between FMA & MSD
2. **Get all metadata** for the matching songs
3. **Find similar metadata** for both datasets
4. **Write code for the similarity model**, deciding what input and output we would have
5. **Create X and y from the .X dataset**
6. **Train the model**

## Setup

In [69]:
# IMPORTS
import json
import os
import numpy as np
import pandas as pd
import sys

import h5py  # for reading MSD .h5 files


from notebook_utils import md, h3, h4, h5
import FMA_code.utils

In [70]:
# PATHS
class paths():
    # General
    DATA_F = 'data/'

    # MSD files
    MSD_F = DATA_F + 'MSD/'
    SIMILAR_TRACKS = MSD_F + 'tracks_with_similar.txt'
    LASTFM_SUBSET_F = MSD_F + 'lastfm_subset/'
    LASTFM_TRAIN_F = MSD_F + 'lastfm_train/'
    LASTFM_TEST_F = MSD_F + 'lastfm_test/'
    MSD_SUBSET_F = MSD_F + 'MillionSongSubset/'
    MSD_SUMMARY = MSD_F + 'msd_summary_file.h5'

    # FMA files
    FMA_F = DATA_F + 'FMA/'
    FMA_METADATA_F = FMA_F + 'fma_metadata/'
    FMA_SMALL_F = FMA_F + 'fma_small/'

In [71]:
# LOAD DATA
# FMA
FMA_tracks = FMA_code.utils.load(paths.FMA_METADATA_F + 'tracks.csv')
FMA_genres = FMA_code.utils.load(paths.FMA_METADATA_F + 'genres.csv')
FMA_features = FMA_code.utils.load(paths.FMA_METADATA_F + 'features.csv')
FMA_echonest = FMA_code.utils.load(paths.FMA_METADATA_F + 'echonest.csv')

# MSD
msd_summary_h5 = h5py.File(paths.MSD_SUMMARY, 'r')

# LAST.FM

## Show all available metadata

In [ ]:
# SHOW METADATA FOR ONE EXAMPLE OF LAST.FM SUBSET
lastfm_ex_path = paths.LASTFM_SUBSET_F + 'A/A/A/TRAAAAW128F429D538.json'
with open(lastfm_ex_path) as f:
    lastfm_ex = json.load(f)

for key in lastfm_ex.keys():
    print(f'{key:<15}', lastfm_ex[key][:2] + ['...'] if isinstance(lastfm_ex[key], list) else lastfm_ex[key])

artist          Casual
timestamp       2011-08-02 20:13:25.674526
similars        [['TRABACN128F425B784', 0.871737], ['TRIAINV12903CB4943', 0.751301], '...']
tags            [['Bay Area', '100'], ['hieroglyiphics', '100'], '...']
track_id        TRAAAAW128F429D538
title           I Didn't Mean To


In [89]:
# SHOW ALL METADATA FOR ONE EXAMPLE OF MSD
msd_ex_path = paths.MSD_SUBSET_F + 'A/X/A/TRAXAAS128F146B741.h5'
h5 = h5py.File(msd_ex_path,'r')
for key in h5.keys():
    print('\n',key)
    for key2 in h5[key].keys():
        print('\t', key2, h5[key][key2].shape)

        # Check if there are extra fields
        fields = h5[key][key2].dtype.fields
        if fields:
            for field in fields:
                print('\t\t', field, h5[key][key2][field])
        else:
            print('\t\t', h5[key][key2][:7])
            # print('\t\t', h5[key][key2].dtype.fields)


 analysis
	 bars_confidence (77,)
		 [0.01  0.406 0.277 0.416 0.534 0.691 0.275]
	 bars_start (77,)
		 [ 0.78155  2.50917  4.22547  5.93918  7.6545   9.36698 11.07402]
	 beats_confidence (311,)
		 [0.928 0.632 0.64  0.407 0.562 0.78  0.665]
	 beats_start (311,)
		 [0.34974 0.78155 1.21612 1.64698 2.07729 2.50917 2.93462]
	 sections_confidence (7,)
		 [1.    0.866 0.437 0.637 0.791 0.879 0.739]
	 sections_start (7,)
		 [  0.        8.08448  37.03275  66.6059   78.64275  91.19099 102.78459]
	 segments_confidence (477,)
		 [0.    1.    1.    1.    1.    0.949 1.   ]
	 segments_loudness_max (477,)
		 [-60.    -10.943  -6.516  -9.45   -6.962  -6.652  -9.828]
	 segments_loudness_max_time (477,)
		 [0.      0.05481 0.06717 0.02053 0.0334  0.03042 0.02525]
	 segments_loudness_start (477,)
		 [-60.    -60.    -46.214 -29.701 -33.415 -20.91  -38.481]
	 segments_pitches (477, 12)
		 [[1.    0.906 0.239 0.2   0.177 0.246 0.268 0.212 0.22  0.226 0.23  0.366]
 [0.076 0.058 0.188 0.168 0.103 0.091 0

In [117]:
# SHOW ALL METADATA KEYS FOR ONE EXAMPLE OF MSD
for key in h5.keys():
    print('\n',key)
    for key2 in h5[key].keys():
        print('\t', key2, h5[key][key2].shape)

        # Check if there are extra fields
        fields = h5[key][key2].dtype.fields
        if fields:
            for field in fields:
                print('\t\t', field, h5[key][key2][field])
        # else:
        #     print('\t\t', h5[key][key2][:7])



 analysis
	 bars_confidence (83,)
	 bars_start (83,)
	 beats_confidence (344,)
	 beats_start (344,)
	 sections_confidence (10,)
	 sections_start (10,)
	 segments_confidence (971,)
	 segments_loudness_max (971,)
	 segments_loudness_max_time (971,)
	 segments_loudness_start (971,)
	 segments_pitches (971, 12)
	 segments_start (971,)
	 segments_timbre (971, 12)
	 songs (1,)
		 analysis_sample_rate [22050]
		 audio_md5 [b'a222795e07cd65b7a530f1346f520649']
		 danceability [0.]
		 duration [218.93179]
		 end_of_fade_in [0.247]
		 energy [0.]
		 idx_bars_confidence [0]
		 idx_bars_start [0]
		 idx_beats_confidence [0]
		 idx_beats_start [0]
		 idx_sections_confidence [0]
		 idx_sections_start [0]
		 idx_segments_confidence [0]
		 idx_segments_loudness_max [0]
		 idx_segments_loudness_max_time [0]
		 idx_segments_loudness_start [0]
		 idx_segments_pitches [0]
		 idx_segments_start [0]
		 idx_segments_timbre [0]
		 idx_tatums_confidence [0]
		 idx_tatums_start [0]
		 key [1]
		 key_confidence

In [ ]:
# SHOW ALL METADATA FOR THE MSD SUMMARY
h5 = h5py.File(paths.MSD_SUMMARY,'r')
for key in h5.keys():
    print('\n',key)
    for key2 in h5[key].keys():
        print('\t', key2, h5[key][key2].shape)

        # Check if there are extra fields
        fields = h5[key][key2].dtype.fields
        if fields:
            for field in fields:
                print('\t\t', field, h5[key][key2][field])
        else:
            print('\t\t', h5[key][key2][:7])
            # print('\t\t', h5[key][key2].dtype.fields)


 analysis
	 songs (1000000,)
		 analysis_sample_rate [22050 22050 22050 ... 22050 22050 22050]
		 audio_md5 [b'aee9820911781c734e7694c5432990ca' b'ed222d07c83bac7689d52753610a513a'
 b'96c7104889a128fef84fa469d60e380c' ...
 b'7d065b833e183244a3c3ed023fcbb70a' b'32473a8e2d20f3efbdcb3caa57d4bf35'
 b'7c4a1f610c8f73d467a1463027a8bc40']
		 danceability [0. 0. 0. ... 0. 0. 0.]
		 duration [252.05506 156.55138 138.97098 ... 553.03791 484.51873 295.07873]
		 end_of_fade_in [2.049 0.258 0.    ... 0.223 0.595 0.   ]
		 energy [0. 0. 0. ... 0. 0. 0.]
		 idx_bars_confidence [0 0 0 ... 0 0 0]
		 idx_bars_start [0 0 0 ... 0 0 0]
		 idx_beats_confidence [0 0 0 ... 0 0 0]
		 idx_beats_start [0 0 0 ... 0 0 0]
		 idx_sections_confidence [0 0 0 ... 0 0 0]
		 idx_sections_start [0 0 0 ... 0 0 0]
		 idx_segments_confidence [0 0 0 ... 0 0 0]
		 idx_segments_loudness_max [0 0 0 ... 0 0 0]
		 idx_segments_loudness_max_time [0 0 0 ... 0 0 0]
		 idx_segments_loudness_start [0 0 0 ... 0 0 0]
		 idx_segments_pitc

In [88]:
# Check % of 0s for danceability & energy
(msd_summary_h5['analysis']['songs']['danceability'] == 0).sum()
(msd_summary_h5['analysis']['songs']['energy'] == 0).sum()

# msd_summary_h5['analysis']['songs']['energy'].sum()

np.int64(1000000)

In [75]:
# NOW GET THE FMA ONE :)
artist_name = 'Chandeliers'
title = 'The Durks'

# Get all related metadata
FMA_tracks.columns
m = (FMA_tracks['artist']['name'] == artist_name) & (FMA_tracks['track']['title'] == title)

pd.set_option('display.max_columns', None)
display(FMA_tracks[m])

track_id = FMA_tracks[m].index[0]
# display(FMA_features[FMA_features.index == track_id])
display(FMA_echonest[FMA_echonest.index == track_id])

album                                                           \
         comments        date_created date_released     engineer favorites   
track_id                                                                     
18805           0 2009-09-16 13:14:52    2009-09-22  Chandeliers         0   

                                                                           \
            id                                        information listens   
track_id                                                                    
18805     4293  <p>Chicago's CHANDELIERS are totally uplifting...    3931   

                                                                         \
             producer tags                  title tracks           type   
track_id                                                                  
18805     Chandeliers   []  Dirty Moves (Sampler)      5  Single Tracks   

                    artist                  \
         active_year_begin active_year_end   
track_id                                     
18805           2004-01-01             NaT   

                                                           \
                                        associated_labels   
track_id                                                    
18805     Captcha Records (HBSP-2X), Pickled Egg (Europe)   

                                                                      \
                                                        bio comments   
track_id                                                               
18805     <p><span style="font-family:Verdana, Geneva, A...        1   

                                                                     \
                date_created favorites   id   latitude     location   
track_id                                                              
18805    2008-11-26 02:05:22         8  100  41.878114  Chicago, IL   

                                                                        \
          longitude                                            members   
track_id                                                                 
18805    -87.629798  Chris Kalis, Harry Brenner, Scott McGaughey, B...   

                                                                          \
                 name                                   related_projects   
track_id                                                                   
18805     Chandeliers  Killer Whales, \nMichael Columbia\nMandate\nMr...   

                                                                 set          \
                   tags             website wikipedia_page     split  subset   
track_id                                                                       
18805     [chandeliers]  thechandeliers.com            NaN  training  medium   

            track                                                      \
         bit_rate comments composer        date_created date_recorded   
track_id                                                                
18805      256000        0      NaN 2009-09-16 13:16:39           NaT   

                                                                       \
         duration favorites   genre_top genres genres_all information   
track_id                                                                
18805          63         2  Electronic   [15]       [15]         NaN   

                                 \
         interest language_code   
track_id                          
18805        1705            en   

                                                                              \
                                                    license listens lyricist   
track_id                                                                       
18805     Attribution-Noncommercial-Share Alike 3.0 Unit...     602      NaN   

                                           
         number publisher tags      title  
track_id                  

echonest                                                   \
         audio_features                                                    
           acousticness danceability   energy instrumentalness  liveness   
track_id                                                                   
18805           0.61392     0.535189  0.53458         0.971656  0.119325   

                                                                               \
                                          metadata                              
         speechiness    tempo   valence album_date album_name artist_latitude   
track_id                                                                        
18805       0.030479  109.976  0.063106        NaN        NaN         30.2676   

                                                                               \
                                                                                
         artist_location artist_longitude  artist_name                release   
track_id                                                                        
18805     Austin, TX, US          -97.743  Chandeliers  Dirty Moves (Sampler)   

                                                                               \
                         ranks                                                  
         artist_discovery_rank artist_familiarity_rank artist_hotttnesss_rank   
track_id                                                                        
18805                      NaN                     NaN                    NaN   

                                                                   \
                                                  social_features   
         song_currency_rank song_hotttnesss_rank artist_discovery   
track_id                                                            
18805                   NaN                  NaN         0.364921   

                                                                             \
                                                                              
         artist_familiarity artist_hotttnesss song_currency song_hotttnesss   
track_id                                                                      
18805              0.400958          0.381724           0.0             0.0   

                                                                              \
         temporal_features                                                     
                       000       001       002       003       004       005   
track_id                                                                       
18805             0.203455  0.572626  0.222005  0.163111  0.208889  0.273975   

                                                                             \
                                                                              
               006       007       008       009       010       011    012   
track_id                                                                      
18805     0.302328  0.585904  0.277035  0.158101  0.256278  0.148349  0.166   

                                                                            \
                                                                             
           013     014    015    016     017    018     019    020     021   
track_id                                                                     
18805     0.67  0.1725  0.084  0.077  0.1565  0.238  0.5525  0.212  0.1015   

                                                                            \
                                                                             
             022     023       024       025       026       027       028   
track_id                                                                     
18805     0.1825  0.0895  0.027242  0.178368  0.030296  0.029767  0.063439   

                                                                              \
               

In [37]:
# Extract titles & artists for MSD tracks that match to FMA
titles = [a.decode('utf-8') for t, a in zip(h5['analysis']['songs']['track_id'], h5['metadata']['songs']['title']) if t.decode('utf-8') in MSD_track_ids]
artists = [a.decode('utf-8') for t, a in zip(h5['analysis']['songs']['track_id'], h5['metadata']['songs']['artist_name']) if t.decode('utf-8') in MSD_track_ids]

assert len(titles) == len(artists) & len(titles) == len(MSD_track_ids)

In [42]:
similars_t_a = {(v['title'], v['artist']) for v in similars_dict.values()}

In [48]:
len([(t, a, tr) for t, a, tr in zip(titles, artists, MSD_track_ids) if (t, a) in similars_t_a])

400

In [ ]:
# MANUALLY SEE IF ANYTHING MATCHES :)

# 1. Get a random title, artist pair from the MSD & get all data from it


## Get matching songs between FMA & LAST.FM

In [6]:
# GET LASTFM-FMA MATCHING TRACKS

# Get title & artist tuple set for FMA
FMA_title_artist_set = set(zip(FMA_tracks[('track', 'title')], FMA_tracks[('artist', 'name')]))

# Get list of tracks with similar tracks on last.fm
with open(paths.SIMILAR_TRACKS) as f:
    tracks_with_similar = f.read().splitlines()

# Get MSD tracks that match FMA & save it in similars_dict
notfound = 0
similars_dict = {}

for track_id in tracks_with_similar:
    rel_track_path = f'{track_id[2]}/{track_id[3]}/{track_id[4]}/{track_id}.json'

    # Get the last.fm folder (train/test) where the track is
    for supfolder in [paths.LASTFM_TEST_F, paths.LASTFM_TRAIN_F]:
        track_path = supfolder + rel_track_path
        if os.path.exists(track_path):
            break
    
    # Extrac the data from the track
    with open(track_path) as f:
        track_dict = json.load(f)

        # Get the title & artist tuple
        title = track_dict['title']
        artist = track_dict['artist']

        # Check if the title & artist match
        if (title, artist) in FMA_title_artist_set:
            similars_dict[track_id] = {
                'similars': track_dict['similars'],
                'artist': track_dict['artist'],
                'title': track_dict['title']   
            }

print(f"FMA and MSD match in {len(similars_dict)} tracks where MSD has similar tracks")

FMA and MSD match in 453 tracks where MSD has similar tracks


In [10]:
# EXTRACT MSD SIMILAR TRACKS
MSD_track_ids = similars_dict.keys()
MSD_track_ids

dict_keys(['TRACCYC128F930B379', 'TRADPJD12903D031C6', 'TRAEJYY128F4218203', 'TRAGYML128F4286683', 'TRAIZBQ128F92F02A7', 'TRAJXWL12903D0537F', 'TRAKKTI128F42B4F25', 'TRALDEB128F428CE7A', 'TRALSLF128F421E8FA', 'TRAMUUZ128F933CA08', 'TRAODHR128F4280D21', 'TRAPJCE128F933EFB6', 'TRAQEZO128F9345243', 'TRAUENB128F9302207', 'TRAULSR128F933DDA6', 'TRAYXFP128F93328B1', 'TRAYYKZ128F148F79E', 'TRBBDLJ128F933B13D', 'TRBBJRH128F92E321D', 'TRBHZIR128F42B9D3F', 'TRBIXYY128F427568E', 'TRBKXPB128F92FB059', 'TRBNTZP12903CB9C0D', 'TRBNYIW128F932D641', 'TRBOHGT128F933DDA1', 'TRBRHZB128F4260D0F', 'TRBRMXW128F4287365', 'TRBRRLH128F147F9B8', 'TRBSGIV128F933EFAE', 'TRBSVSB128F93108D6', 'TRBWOJQ128F933EFAC', 'TRBXKIE128F425BA78', 'TRBYHXS128F42720D9', 'TRCBCCT128F932345A', 'TRCDHJF128F4298A1C', 'TRCDLEX128F9309413', 'TRCFKBD128F93108D0', 'TRCJWFM12903CC9324', 'TRCKKGX12903CD2D06', 'TRCLAGG12903CEE39C', 'TRCLAJZ128F931528E', 'TRCMWIQ128F93167E0', 'TRCQXCN128F933EFB1', 'TRCUAAZ128F933EBB4', 'TRCVJZX128F42553CE',

## Prepare the model's input and output

In [10]:
# PREPARE THE INPUT

In [ ]:
# PREPARE THE OUTPUT